## 導入訓練與測試資料

In [13]:
import numpy as np
import pandas as pd

# show all output
pd.options.display.max_rows = None
pd.options.display.max_columns = None

In [14]:
train_data = pd.read_csv("../data/Train_Cat.csv")
test_data = pd.read_csv("../data/Test_Cat.csv")

train_data = train_data.drop(["Unnamed: 0"], axis=1)
test_data = test_data.drop(["Unnamed: 0"], axis=1)


In [20]:
column = train_data.columns
# replace infinnity data by maximum value in float
train_data = train_data.replace([np.inf, -np.inf], np.finfo(np.float32).max)
test_data = test_data.replace([np.inf, -np.inf], np.finfo(np.float32).max)
train_data.columns = column
test_data.columns = column

In [21]:
y_train = train_data["Cat"]
X_train = train_data.drop(["Cat"], axis=1)
y_test = test_data["Cat"]
X_test = test_data.drop(["Cat"], axis=1)

## 建立具有資料投毒攻擊防禦能力的隨機森林

隨機森林在訓練模型時會從訓練資料集中隨機抽取資料形成子訓練集，這一步驟稱爲Bagging。Bagging可以避免訓練決策樹的時候造成過擬合，減小模型的方差。子資料集會交由不同的決策樹進行訓練，決策樹會隨機選取資料的特徵作爲決策依據，因此每一顆樹的結構都不同。不同的決策樹會生成不同的預測結果，最後使用投票決定最終的預測結果。  
隨機森林進行Bagging的時候，若訓練資料帶有一定比例下的惡意資料，則bagging會將資料可能會將資料分爲帶有惡意資料的自集合和正常資料的子集合，帶有惡意資料的子集合其決策樹也會受影響，受影響的決策樹能夠在投票階段被識別出，最小化惡意資料的影響。

使用其他決策方法取代隨機森林，並比較各決策方法的性能與能耗

In [41]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

class RandomForest():
    def __init__(self,n_estimator=10, max_samples=1.0,
                 max_features=1.0, boostrap=True, bootstrap_features=True, random_state=1):

        '''
        BaggingClassifier:
        base_estimator: 決策方法，如Decision Tree, XGBoost等
        n_estimators: 評估器個數
        max_samples: 從訓練資料集X中抽取的樣本數，用於訓練每個評估器，如果值為int則抽取n個樣本，若爲float則按比例抽取特徵
        max_features: 從訓練資料集X中提取用於訓練每個基本評估器的特徵數，同上
        bootstrap: 是否放回采樣，如果為False則是passing
        bootstrap_features: 是否針對特徵重抽樣
        oob_score: 是否使用oob估計汎化誤差
        random_state: 隨機種子
        '''
        self.classifier = BaggingClassifier(
            DecisionTreeClassifier(random_state=1), n_estimators=n_estimator, max_samples=max_samples,
            max_features=max_features, bootstrap=boostrap, bootstrap_features=bootstrap_features,
            random_state=random_state)
        
    def fit(self, X, y):
        self.classifier.fit(X,y)

    def predict(self, X):
        return self.classifier.predict(X)
    

## 訓練隨機森林
使用經過處理的訓練資料對隨機森林進行訓練

In [39]:
from sklearn import metrics
from sklearn.metrics import classification_report

In [42]:
rf = RandomForest()
rf.fit(X_train, y_train)

In [43]:
train_predict = rf.predict(X_train)

accuracy_train = metrics.accuracy_score(y_train, train_predict)

print("Train Accuracy: {:.4f}%".format(accuracy_train * 100))

Train Accuracy: 99.8880%


In [44]:
print(classification_report(y_train, train_predict))

                   precision    recall  f1-score   support

              DoS       1.00      1.00      1.00     12500
MITM ARP Spoofing       0.99      1.00      1.00     12500
            Mirai       1.00      1.00      1.00     12500
           Normal       1.00      1.00      1.00     50000
             Scan       1.00      1.00      1.00     12500

         accuracy                           1.00    100000
        macro avg       1.00      1.00      1.00    100000
     weighted avg       1.00      1.00      1.00    100000



使用測試資料集驗證模型

In [45]:
test_predict = rf.predict(X_test)

accuracy_test = metrics.accuracy_score(y_test, test_predict)

print("Test Accuracy: {:.4f}%".format(accuracy_test * 100))

Test Accuracy: 97.5380%


In [46]:
print(classification_report(y_test, test_predict))

                   precision    recall  f1-score   support

              DoS       1.00      1.00      1.00      9507
MITM ARP Spoofing       0.75      0.99      0.85      5728
            Mirai       1.00      0.97      0.98     66238
           Normal       0.95      1.00      0.97      6434
             Scan       0.99      0.98      0.98     12093

         accuracy                           0.98    100000
        macro avg       0.94      0.99      0.96    100000
     weighted avg       0.98      0.98      0.98    100000

